In [1]:
library(readxl)
library(plyr)

In [2]:
#load in the Excel file from IMGT
imgt <- read_excel("vquest.xls",sheet = 1)

In [3]:
#split into a Chain and b chain
achain <- imgt[grep("JMWA",imgt$`Sequence ID`),]
bchain <- imgt[grep("JMWB",imgt$`Sequence ID`),]

In [4]:
#get only these columns from the excel file
interestA <- c("Sequence ID","V-GENE and allele","J-GENE and allele","AA JUNCTION")
interestB <- c("Sequence ID","V-GENE and allele","J-GENE and allele","D-GENE and allele","AA JUNCTION")

In [5]:
#make a dataframe from columns of interest
dfa<- data.frame(achain[,interestA])
dfb<- data.frame(bchain[,interestB])

In [6]:
#Get the well number
dfa$number <- sapply(strsplit(dfa[,1],"-"), "[", 3)
dfb$number <- sapply(strsplit(dfb[,1],"-"), "[", 3)

In [7]:
#order by well number for pairing
dfa <- dfa[order(as.numeric(gsub("([0-9]+)([A-Z]+)", "\\1", dfa$number)),gsub("([0-9]+)([A-Z]+)", "\\2", dfa$number)),]
dfb <- dfb[order(as.numeric(gsub("([0-9]+)([A-Z]+)", "\\1", dfb$number)),gsub("([0-9]+)([A-Z]+)", "\\2", dfb$number)),]

In [8]:
#give order for later sorting 
dfa$order <- 1:nrow(dfa)
dfb$order <- 1:nrow(dfb)

In [9]:
library(stringr)

In [10]:
#retrieve only the chain form
clean <- function(region){
    t1 <- gsub("\\s*\\([^\\)]+\\)","",as.character(region))
    t2 <- str_replace(t1, "F", "")
    t3 <- gsub("Homsap", "",t2)
    t4 <- gsub(" |, ", "",t3)
    return(t4)
}

In [11]:
#Get only the amino acids(remove any notes/comments)
cleanAA <- function(region){
    t1 <- gsub("\\s*\\([^\\)]+\\)","",as.character(region))
    t6 <- gsub(" |, ", "",t1)
    return(t6)
}

In [12]:
#get coordinates for matching
dfa$genes <- paste(clean(dfa$V.GENE.and.allele),clean(dfa$J.GENE.and.allele),cleanAA(dfa$AA.JUNCTION), sep = ":")
dfb$genes <- paste(clean(dfb$V.GENE.and.allele),clean(dfb$J.GENE.and.allele),clean(dfb$D.GENE.and.allele),cleanAA(dfb$AA.JUNCTION), sep = ":")

In [13]:
#get the frequency of each of the genes 
freqa <- count(dfa$genes)
freqb <- count(dfb$genes)

In [14]:
freqa$group <- 1:nrow(freqa)
freqb$group <- 1:nrow(freqb)

In [15]:
dfa <- merge(dfa,freqa,by.x='genes', by.y=1)
dfb <- merge(dfb,freqb,by.x='genes', by.y=1)

In [17]:
groupings <- function(df){
    grouped <- list()
    number <- df$number 
    names(number) <- df$group
    for(i in 1:length(unique(df$group))){
        grouped[[i]]<- paste(number[names(number) == i], collapse=",")
    }
    df <- data.frame(row.names  = 1:length(unique(df$group)) ,unlist(grouped))
    return(df)
}

In [18]:
pair_groupings <- function(df){
    grouped <- list()
    numbera <- df$number.x
    names(numbera) <- df$group
    numberb <- df$number.y 
    names(numberb) <- df$group
    for(i in 1:length(unique(df$group))){
        grouped[[i]]<- paste(paste(numbera[names(numbera) == i], collapse=","),paste(numberb[names(numberb) == i], collapse=","),collapse = ",")
    }
    df <- data.frame(row.names  = 1:length(unique(df$group)) ,unlist(grouped))
    return(df)
}

In [19]:
groupinga <- groupings(dfa)
groupingb <- groupings(dfb)

In [20]:
dfa <- merge(dfa,groupinga,by.x='group', by.y=0)
dfb <- merge(dfb,groupingb,by.x='group', by.y=0)

In [21]:
merged <- merge(dfa,dfb, by ="order",all = T)

In [23]:
merged$pair <- paste(merged$genes.x,merged$genes.y, sep = ":")

In [24]:
freqab <- count(merged$pair)

In [25]:
freqab$group <- 1:nrow(freqab)

In [26]:
merged <- merge(merged,freqab,by.x='pair', by.y=1)

In [27]:
groupingab <- pair_groupings(merged)

In [28]:
merged <- merge(merged,groupingab,by.x='group', by.y=0)

In [29]:
final <- merged[!duplicated(merged$group),-(grep("genes|order|pair|number|group.x|group.y",colnames(merged)))]

In [30]:
final <- final[order(final$freq,decreasing = T),-1]

In [31]:
colnames(final) <- c("Sequence_ID_Alpha", "Alpha_V","Alpha_J","Alpha_CDR3","Frequency_of_A","Same_A_Wells","Sequence_ID_Beta", "Beta_V","Beta_J","Beta_D","Beta_CDR3","Frequency_of_B","Same_B_Wells","Frequency_of_Pair","Same_Pair_Wells")

In [32]:
final

,Sequence_ID_Alpha,Alpha_V,Alpha_J,Alpha_CDR3,Frequency_of_A,Same_A_Wells,Sequence_ID_Beta,Beta_V,Beta_J,Beta_D,Beta_CDR3,Frequency_of_B,Same_B_Wells,Frequency_of_Pair,Same_Pair_Wells
,<chr>,<chr>,<chr>,<chr>,<int>,<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<fct>,<int>,<fct>
6,13-05-1E-JMWAC2_E01,Homsap TRAV12-3*01 F,Homsap TRAJ30*01 F,CAMNGDDKIIF,4,"1E,1F,3D,5G",13-06-2E-JMWBC2_F01,Homsap TRBV7-2*04 (F),Homsap TRBJ1-2*01 F,Homsap TRBD1*01 F,CASSLPDRINYGYTF,3,"2E,6G,4D",3,"1E,5G,3D 2E,6G,4D"
1,NA,NA,NA,NA,NA,NA,13-37-12C-JMWBC2_E05,Homsap TRBV10-3*01 F,Homsap TRBJ2-7*01 F,Homsap TRBD2*01 F,CAISEIIAHFPYEQYF,1,12C,1,NA 12C
2,13-27-7E-JMWAC2_C04,Homsap TRAV1-2*01 F,Homsap TRAJ30*01 F,CAV*#NRDDKIIF,1,7E,13-28-8E-JMWBC2_D04,"Homsap TRBV7-2*01 F, or Homsap TRBV7-2*04 (F)",Homsap TRBJ1-2*01 F,Homsap TRBD1*01 F,CASSLPDRINYGYTF,1,8E,1,7E 8E
3,13-17-5B-JMWAC2_A03,Homsap TRAV12-1*01 F,Homsap TRAJ4*01 F,CVVNWFSGGYNKLIF,2,"5B,3F",13-18-6B-JMWBC2_B03,Homsap TRBV11-2*01 F,Homsap TRBJ1-1*01 F,Homsap TRBD1*01 F,CASSLPTGGALNTEAFF,2,"6B,8A",1,5B 6B
4,13-15-3F-JMWAC2_G02,Homsap TRAV12-1*01 F,Homsap TRAJ4*01 F,CVVNWFSGGYNKLIF,2,"5B,3F",13-16-4F-JMWBC2_H02,Homsap TRBV28*01 F,Homsap TRBJ1-2*01 F,Homsap TRBD2*01 F,CASSTTRDWPLDGYTF,1,4F,1,3F 4F
5,13-07-1F-JMWAC2_G01,Homsap TRAV12-3*01 F,Homsap TRAJ30*01 F,CAMNGDDKIIF,4,"1E,1F,3D,5G",13-08-2F-JMWBC2_H01,"Homsap TRBV7-2*01 F, or Homsap TRBV7-2*04 (F)",Homsap TRBJ1-2*01 F,Homsap TRBD1*01 F,CASSLPDRIKYGYTF,1,2F,1,1F 2F
9,13-21-5D-JMWAC2_E03,Homsap TRAV19*01 F,Homsap TRAJ54*01 F,CALSLIYQGAQKLVF,1,5D,13-22-6D-JMWBC2_F03,Homsap TRBV5-4*04 (F) (see comment),Homsap TRBJ2-5*01 F,NA,MCQQVR*REALF (see V-DOMAIN Functionality comment),1,6D,1,5D 6D
10,13-11-3B-JMWAC2_C02,Homsap TRAV2*01 F,Homsap TRAJ26*01 F,CAVGPNYGQNFVF,1,3B,13-12-4B-JMWBC2_D02,Homsap TRBV4-3*01 F,Homsap TRBJ2-3*01 F,Homsap TRBD1*01 F,CASSQVPTGGPDTQYF,1,4B,1,3B 4B
11,13-25-7A-JMWAC2_A04,Homsap TRAV27*01 F,Homsap TRAJ20*01 F,CAGITNDYKLSF,1,7A,13-26-8A-JMWBC2_B04,Homsap TRBV11-2*01 F,Homsap TRBJ1-1*01 F,Homsap TRBD1*01 F,CASSLPTGGALNTEAFF,2,"6B,8A",1,7A 8A


In [33]:
#write.table(final,file = outfile,row.names = F, quote = F, sep ="\t")